In [ ]:
!pip install transformers

In [ ]:
import torch
import sys

In [ ]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
data=open('/content/drive/MyDrive/para_kqc_sim_data.txt','r',encoding='utf-8')
lines=data.readlines()

print(lines[0:10])

['메일을 다 비울까 아니면 안읽은 것만 지울까?\t메일 중에 안읽은 것만 지울까? 다 지울까?\t1\n', '메일을 다 비울까 아니면 안읽은 것만 지울까?\t안읽은 메일만 지워 다지워?\t1\n', '메일을 다 비울까 아니면 안읽은 것만 지울까?\t다 지울까 안읽은 메일만 지울까?\t1\n', '메일을 다 비울까 아니면 안읽은 것만 지울까?\t전체를 비울까 안읽은 것만 비울까?\t1\n', '메일을 다 비울까 아니면 안읽은 것만 지울까?\t안읽은 메일만 지울꺼야? 아니면 다 지울꺼야?\t1\n', '메일을 다 비울까 아니면 안읽은 것만 지울까?\t어떻게 지울까? 안읽은거만? 전체 다?\t1\n', '메일을 다 비울까 아니면 안읽은 것만 지울까?\t메일을 다 지울지 안읽은거만 지울지 알려주세요\t1\n', '메일을 다 비울까 아니면 안읽은 것만 지울까?\t메일은 다 지울수도 있고, 안읽은거만 지울 수도 있어. 어떻게 할래?\t1\n', '메일을 다 비울까 아니면 안읽은 것만 지울까?\t안읽은 메일만 지우든가, 다 지울 수 있는데 어떻게 할꺼야?\t1\n', '지메일 쓸래, 네이버 메일 쓸래\t지메일을 쓸거야 네이버 메일을 쓸꺼야?\t1\n']


In [ ]:
import random
random.shuffle(lines)

In [ ]:
# train,test 8:2로 분리
train={'sent_a':[], 'sent_b':[], 'label':[]}
test={'sent_a':[], 'sent_b':[], 'label':[]}
for i, line in enumerate(lines):
  if i<len(lines)*0.8:
    line=line.strip()
    train['sent_a'].append(line.split('\t')[0])
    train['sent_b'].append(line.split('\t')[1])
    train['label'].append(int(line.split('\t')[2]))
  else:
    line=line.strip()
    test['sent_a'].append(line.split('\t')[0])
    test['sent_b'].append(line.split('\t')[1])
    test['label'].append(int(line.split('\t')[2]))


In [ ]:
import pandas as pd

In [ ]:
train_data=pd.DataFrame({'sent_a':train['sent_a'],'sent_b':train['sent_b'],'label':train['label']})
test_data=pd.DataFrame({'sent_a':test['sent_a'],'sent_b':test['sent_b'],'label':test['label']})

In [ ]:
# 데이터 중복을 제외한 개수 확인
print('학습데이터:',train_data.groupby(['sent_a','sent_b']).ngroups,'라벨:',train_data['label'])
print('테스트데이터:',test_data.groupby(['sent_a','sent_b']).ngroups,'라벨:',test_data['label'])

# 중복 데이터 제거
train_data.drop_duplicates(subset=['sent_a','sent_b'],inplace=True)
test_data.drop_duplicates(subset=['sent_a','sent_b'],inplace=True)

print(len(train_data))
print(len(test_data))

학습데이터: 15171 라벨: 0        1
1        0
2        0
3        0
4        0
        ..
15167    1
15168    1
15169    1
15170    1
15171    1
Name: label, Length: 15172, dtype: int64
테스트데이터: 3791 라벨: 0       0
1       0
2       1
3       0
4       1
       ..
3787    0
3788    0
3789    0
3790    0
3791    0
Name: label, Length: 3792, dtype: int64
15171
3791


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# null 데이터 제거
train_data.replace('',np.nan, inplace=True)
test_data.replace('',np.nan, inplace=True)

train_data=train_data.dropna(how='any')
test_data=test_data.dropna(how='any')

print(len(train_data))
print(len(test_data))

15171
3791


In [ ]:
print(train_data['sent_a'][0])
print(train_data['sent_b'][0])
print(train_data['label'][0])

배달의 민족 결제한 거 메일로 받으려면 어떻게 해?
배달의 민족 결제 내역은 어떤 방식으로 메일로 받을 수 있나요?
1


In [ ]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME='bert-base-multilingual-cased'
tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
tokenized_train_sentences=tokenizer(
    list(train_data['sent_a'][0:]),
    list(train_data['sent_b'][0:]),
    return_tensors='pt',
    padding=True,
    truncation=True,
    add_special_tokens=True,
    max_length=64
)

In [ ]:
tokenized_test_sentences=tokenizer(
    list(test_data['sent_a'][0:]),
    list(test_data['sent_b'][0:]),
    return_tensors='pt',
    padding=True,
    truncation=True,
    add_special_tokens=True,
    max_length=64
)

In [ ]:
train_label=train_data['label'].values[0:]
test_label=test_data['label'].values[0:]

In [ ]:
class MultiSentDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings=encodings
    self.labels=labels

  def __getitem__(self,idx):
    item={key: torch.tensor(val[idx] for key, val in self.encodings.items())}
    item['lables']=torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

In [ ]:
train_dataset=MultiSentDataset(tokenized_train_sentences, train_label)
test_dataset=MultiSentDataset(tokenized_test_sentences, test_label)

In [ ]:
!pip install transformers==4.30

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertConfig

training_args=TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=500,
    save_total_limit=2
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
model=BertForSequenceClassification.from_pretrained(MODEL_NAME,num_labels=2)
model.parameters
model.to(device)